In [9]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math

In [3]:
mr_file = "../data/mr.txt"

In [4]:
df = pd.read_csv(mr_file, sep=";")

In [12]:
def extract_cpath(cpath):
    if (isinstance(cpath, float)):
        return []
    if (cpath ==''):
        return []
    return [int(s) for s in cpath.split(',')]

In [13]:
df["cpath_list"] = df.apply(lambda row: extract_cpath(row.cpath), axis=1)

In [14]:
all_edge_ids = np.unique(np.hstack(df.cpath_list)).tolist()

In [16]:
network_gdf = gpd.read_file("../data/porto/edges.shp")
network_gdf.id = network_gdf.fid.astype(int)
network_gdf.head()

/home/wms/anaconda3/envs/wms/lib/python3.6/site-packages/geopandas/geodataframe.py:182: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


,osmid,oneway,name,highway,length,ref,bridge,junction,lanes,maxspeed,access,tunnel,width,service,area,u,v,key,fid,geometry
0,298888962,True,Avenida Vasco da Gama,residential,56.521,None,None,None,None,None,None,None,None,None,None,3028549663,3028549666,0,0,"LINESTRING (-8.61782 41.22851, -8.61778 41.228..."
1,298888969,False,Avenida Vasco da Gama,residential,15.502,None,None,None,None,None,None,None,None,None,None,3028549663,295232574,0,1,"LINESTRING (-8.61782 41.22851, -8.61786 41.228..."
2,298888962,True,Avenida Vasco da Gama,residential,34.347,None,None,None,None,None,None,None,None,None,None,3028549666,3028549668,0,2,"LINESTRING (-8.61732 41.22881, -8.61701 41.22902)"
3,298888971,False,Avenida Vasco da Gama,residential,6.977,None,None,None,None,None,None,None,None,None,None,3028549666,3028549667,0,3,"LINESTRING (-8.61732 41.22881, -8.61737 41.22885)"
4,298888971,False,Avenida Vasco da Gama,residential,6.977,None,None,None,None,None,None,None,None,None,None,3028549667,3028549666,0,4,"LINESTRING (-8.61737 41.22885, -8.61732 41.22881)"


In [17]:
edges_df = network_gdf[network_gdf.id.isin(all_edge_ids)].reset_index()

In [18]:
edges_df["points"] = edges_df.apply(lambda row: len(row.geometry.coords), axis=1)

In [19]:
edges_df.head()


,index,osmid,oneway,name,highway,length,ref,bridge,junction,lanes,...,tunnel,width,service,area,u,v,key,fid,geometry,points
0,0,298888962,True,Avenida Vasco da Gama,residential,56.521,None,None,None,None,...,None,None,None,None,3028549663,3028549666,0,0,"LINESTRING (-8.61782 41.22851, -8.61778 41.228...",5
1,1,298888969,False,Avenida Vasco da Gama,residential,15.502,None,None,None,None,...,None,None,None,None,3028549663,295232574,0,1,"LINESTRING (-8.61782 41.22851, -8.61786 41.228...",3
2,2,298888962,True,Avenida Vasco da Gama,residential,34.347,None,None,None,None,...,None,None,None,None,3028549666,3028549668,0,2,"LINESTRING (-8.61732 41.22881, -8.61701 41.22902)",2
3,5,298888964,True,Avenida Vasco da Gama,residential,54.853,None,None,None,None,...,None,None,None,None,3028549667,3028549663,0,5,"LINESTRING (-8.61737 41.22885, -8.61741 41.228...",5
4,6,128006483,False,Rua de São Miguel,residential,8.870,None,None,None,None,...,None,None,None,None,3028549668,295232577,0,6,"LINESTRING (-8.61701 41.22902, -8.61706 41.22909)",2


In [21]:
edges_df[["fid","u","v","geometry","points"]].to_csv("../data/matched_edges.csv",sep=";",index=False)